# Домашнее задание по теме «Python для анализа данных: pandas»
Преподаватель: Константин Башевой

**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [1]:
import pandas as pd

In [2]:
geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [3]:
df = pd.read_csv(r'Python_для анализа данных pandas\keywords.csv')

In [4]:
def get_FO(chk_str):    
    for FO,cities in geo_data.items():
        for city in cities:
            if city in chk_str['keyword']:
                return FO
    return 'undefined'

In [5]:
df['region'] = df.apply(get_FO, axis = 1)

In [6]:
df.groupby(['region']).agg(['count'])['shows']

,count
region,
undefined,99253
Дальний Восток,94
Северо-Запад,266
Центр,387


**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [7]:
df1 = pd.read_csv(r'Python_для анализа данных pandas\ml-latest-small\ratings.csv')

In [8]:
def rating(x):
    chk = x['rating']
    if chk <= 2:
        res = 'низкий рейтинг'
    elif chk <= 4:
        res = 'средний рейтинг'
    else:
        res = 'высокий рейтинг'
    return res

In [9]:
df1['class'] = df1.apply(rating,axis=1)
df1.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [10]:
ratings = pd.read_csv(r'Python_для анализа данных pandas\ml-latest-small\ratings.csv')

In [11]:
grouped = ratings.groupby('userId')['timestamp'].agg(['count','min','max'])#.rename(columns={'count':'cnt'})

In [12]:
filtred = grouped[grouped['count'] >= 100]

In [13]:
filtred['Lifetime'] = filtred.apply(lambda x: x['max']-x['min'],axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
str(int(filtred.Lifetime.mean()/3600/24)) + ' дней средний Lifetime киноманов'

'455 дней средний Lifetime киноманов'

**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [15]:
years = [i for i in range(1950,2011)]

In [16]:
movies = pd.read_csv(r'Python_для анализа данных pandas\ml-latest-small\movies.csv')

In [17]:
def production_years(row):
    for y in years:
        if str(y) in row['title']:
            return y
    return 1900

In [18]:
movies['year'] = movies.apply(production_years,axis=1)

In [19]:
joined = movies.merge(ratings, on = 'movieId',how = 'left')

In [23]:
better_days = joined.groupby('year').mean()[['rating']].reset_index().sort_values('rating',ascending = False)
better_days.head()

,year,rating
8,1957,4.014241
23,1972,4.011136
3,1952,4.000000
5,1954,3.994220
2,1951,3.983539
